In [ ]:
#Loading in Packages and Data

#Importing Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
from matplotlib.gridspec import GridSpec
from matplotlib.lines import Line2D
import xarray as xr
import os; import time
import pickle
import h5py
###############################################################
def coefs(coefficients,degree):
    coef=coefficients
    coefs=""
    for n in range(degree, -1, -1):
        string=f"({coefficients[len(coef)-(n+1)]:.1e})"
        coefs+=string + f"x^{n}"
        if n != 0:
            coefs+=" + "
    return coefs
###############################################################

# Importing Model Data
check=False
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'

# # dx = 1 km; Np = 1M; Nt = 5 min
# data1=xr.open_dataset(dir+'../../cm1r20.3/run/cm1out_1km_5min.nc', decode_timedelta=True) #***
# parcel1=xr.open_dataset(dir+'../../cm1r20.3/run/cm1out_pdata_1km_5min_1e6.nc', decode_timedelta=True) #***
# res='1km';t_res='5min'
# Np_str='1e6'

# # dx = 1km; Np = 50M; Nz = 95
# #Importing Model Data
# dir2='/home/air673/koa_scratch/'
# data1=xr.open_dataset(dir2+'cm1out_1km_1min_95nz.nc', decode_timedelta=True) #***
# parcel1=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_95nz.nc', decode_timedelta=True) #***
# res='1km'; t_res='1min_95nz'; Np_str='50e6'

# dx = 250m; Np = 50M
# Importing Model Data
dir2='/home/air673/koa_scratch/'
data1=xr.open_dataset(dir2+'cm1out_250m_1min_50M.nc', decode_timedelta=True) #***
parcel1=xr.open_dataset(dir2+'cm1out_pdata_250m_1min_50M.nc', decode_timedelta=True) #***
res='250m'; t_res='1min'; Np_str='50e6'

In [ ]:
times=data1['time'].values/(1e9 * 60); times=times.astype(float);
minutes=1/times[1] #1 / minutes per timestep = timesteps per minute
job_array=False;index_adjust=0
ocean_fraction=2/8

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
path=dir2+'Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions

# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

#####

#Import StatisticalFunctions 
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
path=dir2+'Functions/'
sys.path.append(path)

import StatisticalFunctions
from StatisticalFunctions import * # import NumericalFunctions 

In [ ]:
#CL VS NONCL
################################################################

In [ ]:
##########################################################
#PLOTTING

In [ ]:
#LIMITING Y AXIS
limit_y=True
# limit_y=False

In [ ]:
def averaged_profiles(profile):
    out_var = profile[(profile[:, 1] > 1)]  # gets rid of rows that have no data
    out_var = np.array([out_var[:, 0] / out_var[:, 1], out_var[:, 2]]).T  # divides the data column by the counter column
    return out_var

In [ ]:
def LoadAllCloudBase():
    dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/OUTPUT/'
    in_file = dir2 + f"all_cloudbase_{res}_{t_res}_{Np_str}.pkl"
    with open(in_file, 'rb') as f:
        all_cloudbase = pickle.load(f)
    return(all_cloudbase)
min_all_cloudbase=np.nanmin(LoadAllCloudBase())
all_cloudbase=min_all_cloudbase
print(f"Minimum Cloudbase is: {all_cloudbase}\n")

In [ ]:
def LoadMeanLFC():
    dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/OUTPUT/'
    in_file = dir2 + f"MeanLFC_{res}_{t_res}_{Np_str}.pkl"
    with open(in_file, 'rb') as f:
        MeanLFC = pickle.load(f)
    return MeanLFC
MeanLFC=LoadMeanLFC()
print(f"Mean LFC is: {MeanLFC}\n")

In [ ]:
#OLD UNCOMBINED VERSION

# def PlotProfiles(ax,type1,type2,type3,type4):
#     if type3!="NET":
#         min_value=0
#     else:
#         min_value=None
#     lw = 0.8
    
#     # cutoff_height = 100 #doesn't do anything since ztop is much less than 100
#     # z_cutoff = np.where(globals()[f"{type1}_ALL_profile_array_{type3}_{type4}"][:, 2] <= cutoff_height)[0][-1]

#     profiles = [
#         (globals()[f"{type1}_ALL_profile_array_{type3}_{type4}"], globals()[f"{type1}_SHALLOW_profile_array_{type3}_{type4}"], globals()[f"{type1}_DEEP_profile_array_{type3}_{type4}"], type1),
#         (globals()[f"{type2}_ALL_profile_array_{type3}_{type4}"], globals()[f"{type2}_SHALLOW_profile_array_{type3}_{type4}"], globals()[f"{type2}_DEEP_profile_array_{type3}_{type4}"], type2)
#     ]

#     SE_profiles = [
#         (globals()[f"{type1}_ALL_profile_array_{type3}_{type4}_SE"], globals()[f"{type1}_SHALLOW_profile_array_{type3}_{type4}_SE"], globals()[f"{type1}_DEEP_profile_array_{type3}_{type4}_SE"], type1),
#         (globals()[f"{type2}_ALL_profile_array_{type3}_{type4}_SE"], globals()[f"{type2}_SHALLOW_profile_array_{type3}_{type4}_SE"], globals()[f"{type2}_DEEP_profile_array_{type3}_{type4}_SE"], type2)
#     ]
    

#     for ((ALL_profile_array, SHALLOW_profile_array, DEEP_profile_array, label),
#      (ALL_SE_array, SHALLOW_SE_array, DEEP_SE_array, _)) in zip(profiles, SE_profiles):

#         # ALL_profile = averaged_profiles(ALL_profile_array)[:z_cutoff]
#         # SHALLOW_profile = averaged_profiles(SHALLOW_profile_array)[:z_cutoff]
#         # DEEP_profile = averaged_profiles(DEEP_profile_array)[:z_cutoff]
#         ALL_profile = averaged_profiles(ALL_profile_array)
#         SHALLOW_profile = averaged_profiles(SHALLOW_profile_array)
#         DEEP_profile = averaged_profiles(DEEP_profile_array)

#         if label == type1:
#             ax.plot(ALL_profile[:, 0], ALL_profile[:, -1], linestyle='solid',color='black', label=f'ALL ({type1})', lw=lw)
#             PlotSE(ax,ALL_profile,ALL_SE_array,color='black',factor=factor, min_value=min_value)
#             ax.plot(SHALLOW_profile[:, 0], SHALLOW_profile[:, -1], linestyle='solid', color='green', label=f'SHALLOW ({type1})', lw=lw)
#             PlotSE(ax,SHALLOW_profile,SHALLOW_SE_array,color='green',factor=factor, min_value=min_value)
#             ax.plot(DEEP_profile[:, 0], DEEP_profile[:, -1], linestyle='solid',color='blue', label=f'DEEP ({type1})', lw=lw)
#             PlotSE(ax,DEEP_profile,DEEP_SE_array,color='blue',factor=factor, min_value=min_value)
#         else:
#             ax.plot(ALL_profile[:, 0], ALL_profile[:, -1], linestyle='dashed', color='black', label=f'ALL ({type2})', lw=lw)
#             PlotSE(ax,ALL_profile,ALL_SE_array,color='black',factor=factor, min_value=min_value)
#             ax.plot(SHALLOW_profile[:, 0], SHALLOW_profile[:, -1], linestyle='dashed', color='green', label=f'SHALLOW ({type2})', lw=lw)
#             PlotSE(ax,SHALLOW_profile,SHALLOW_SE_array,color='green',factor=factor, min_value=min_value)
#             ax.plot(DEEP_profile[:, 0], DEEP_profile[:, -1], linestyle='dashed', color='blue', label=f'DEEP ({type2})', lw=lw)
#             PlotSE(ax,DEEP_profile,DEEP_SE_array,color='blue',factor=factor, min_value=min_value)

#         apply_scientific_notation([ax])

#     ax.set_xlabel(r"($kg m^{-3} s^{-1}$)")  
#     ax.set_ylabel('z (km)')
#     ax.grid(True)
#     ax.axvline(0, linestyle='dashed', color='grey', zorder=-10)
#     ax.axhline(all_cloudbase, color='purple', linestyle='dashed')
#     ax.axhline(MeanLFC/1000,color='green',linestyle='dashed')
#     ax.legend(fontsize=8)

In [ ]:
def PlotSE(ax, profile, SE_profile, color, factor, switch=1, alpha=0.05, min_value=None):
    lower = profile[:, 0] - factor * SE_profile[:, 0] * switch
    upper = profile[:, 0] + factor * SE_profile[:, 0] * switch
    if min_value is not None:
        lower = np.maximum(lower, min_value)
    ax.fill_betweenx(profile[:, -1], lower, upper, color=color, alpha=alpha)
    #Example #PlotSE(ax,ALL_profile,ALL_SE_array,color=color,factor=factor, min_value=min_value)
lw=0.7
def PlotProfiles(ax, types, line_styles, type3, type4,lw=lw):
    min_value = 0 if type3 != "NET" else None

    for t, linestyle in zip(types, line_styles):
        # Grab the global arrays dynamically
        ALL_profile_array = globals()[f"{t}_ALL_profile_array_{type3}_{type4}"]
        SHALLOW_profile_array = globals()[f"{t}_SHALLOW_profile_array_{type3}_{type4}"]
        DEEP_profile_array = globals()[f"{t}_DEEP_profile_array_{type3}_{type4}"]

        ALL_SE_array = globals()[f"{t}_ALL_profile_array_{type3}_{type4}_SE"]
        SHALLOW_SE_array = globals()[f"{t}_SHALLOW_profile_array_{type3}_{type4}_SE"]
        DEEP_SE_array = globals()[f"{t}_DEEP_profile_array_{type3}_{type4}_SE"]

        ALL_profile = averaged_profiles(ALL_profile_array)
        SHALLOW_profile = averaged_profiles(SHALLOW_profile_array)
        DEEP_profile = averaged_profiles(DEEP_profile_array)

        # Color scheme (can customize as needed)
        colors = {'ALL': 'black', 'SHALLOW': 'green', 'DEEP': 'blue'}

        # ax.plot(ALL_profile[:, 0], ALL_profile[:, -1], linestyle=linestyle,
        #         color=colors['ALL'], label=f'ALL ({t})', lw=lw)
        # PlotSE(ax, ALL_profile, ALL_SE_array, color=colors['ALL'], factor=factor, min_value=min_value)

        if t == "SBZ": t = "SBF" #*#* changing SBZ to SBF
        ax.plot(SHALLOW_profile[:, 0], SHALLOW_profile[:, -1], linestyle=linestyle,
                color=colors['SHALLOW'], label=f'SHALLOW ({t})', lw=lw)
        PlotSE(ax, SHALLOW_profile, SHALLOW_SE_array, color=colors['SHALLOW'], factor=factor, min_value=min_value)

        ax.plot(DEEP_profile[:, 0], DEEP_profile[:, -1], linestyle=linestyle,
                color=colors['DEEP'], label=f'DEEP ({t})', lw=lw)
        PlotSE(ax, DEEP_profile, DEEP_SE_array, color=colors['DEEP'], factor=factor, min_value=min_value)

    
    SnapLimitsToTicks([ax], dim='x')
    apply_scientific_notation([ax],decimals=2,scientific=True)
    ax.set_xlabel(r"($kg m^{-3} s^{-1}$)")  
    ax.set_ylabel('z (km)')
    ax.grid(True)
    ax.legend(fontsize=8)

In [ ]:
def PlotTwo(ax, types, line_styles, factor=1, lw=lw):
    min_value = 0

    # Color scheme
    colors = {'ALL': 'black', 'SHALLOW': 'green', 'DEEP': 'blue'}

    for t, linestyle in zip(types, line_styles):
        # Dynamically get arrays using globals()
        ALL_array = globals()[f"{t}_ALL_profile_array_G_TO_C_E"]
        SHALLOW_array = globals()[f"{t}_SHALLOW_profile_array_G_TO_C_E"]
        DEEP_array = globals()[f"{t}_DEEP_profile_array_G_TO_C_E"]

        ALL_SE = globals()[f"{t}_ALL_profile_array_G_TO_C_E_SE"]
        SHALLOW_SE = globals()[f"{t}_SHALLOW_profile_array_G_TO_C_E_SE"]
        DEEP_SE = globals()[f"{t}_DEEP_profile_array_G_TO_C_E_SE"]

        # Average profiles
        ALL_profile = averaged_profiles(ALL_array)
        SHALLOW_profile = averaged_profiles(SHALLOW_array)
        DEEP_profile = averaged_profiles(DEEP_array)

        # Plot each profile + shaded SE
        # ax.plot(ALL_profile[:, 0], ALL_profile[:, -1], linestyle=linestyle,
        #         color=colors['ALL'], label=f'ALL ({t})', lw=lw)
        # PlotSE(ax, ALL_profile, ALL_SE, color=colors['ALL'], factor=factor, min_value=min_value)

        ax.plot(SHALLOW_profile[:, 0], SHALLOW_profile[:, -1], linestyle=linestyle,
                color=colors['SHALLOW'], label=f'SHALLOW ({t})', lw=lw)
        PlotSE(ax, SHALLOW_profile, SHALLOW_SE, color=colors['SHALLOW'], factor=factor, min_value=min_value)

        ax.plot(DEEP_profile[:, 0], DEEP_profile[:, -1], linestyle=linestyle,
                color=colors['DEEP'], label=f'DEEP ({t})', lw=lw)
        PlotSE(ax, DEEP_profile, DEEP_SE, color=colors['DEEP'], factor=factor, min_value=min_value)

    SnapLimitsToTicks([ax], dim='x')
    apply_scientific_notation([ax],decimals=2,scientific=True)
    ax.set_xlabel(r"($kg m^{-3} s^{-1}$)")
    ax.set_ylabel('z (km)')
    ax.grid(True)
    ax.legend(fontsize=8)
    return ax

def PlotThree(ax, types, line_styles, factor=1, lw=lw):
    min_value = 0

    # Color scheme
    colors = {'ALL': 'black', 'SHALLOW': 'green', 'DEEP': 'blue'}

    for t, linestyle in zip(types, line_styles):
        # Dynamically get arrays using globals()
        ALL_array = globals()[f"{t}_ALL_profile_array_G_TO_C_D"]
        SHALLOW_array = globals()[f"{t}_SHALLOW_profile_array_G_TO_C_D"]
        DEEP_array = globals()[f"{t}_DEEP_profile_array_G_TO_C_D"]

        ALL_SE = globals()[f"{t}_ALL_profile_array_G_TO_C_D_SE"]
        SHALLOW_SE = globals()[f"{t}_SHALLOW_profile_array_G_TO_C_D_SE"]
        DEEP_SE = globals()[f"{t}_DEEP_profile_array_G_TO_C_D_SE"]

        # Average profiles
        ALL_profile = averaged_profiles(ALL_array)
        SHALLOW_profile = averaged_profiles(SHALLOW_array)
        DEEP_profile = averaged_profiles(DEEP_array)

        # Plot each profile + shaded SE
        # ax.plot(ALL_profile[:, 0], ALL_profile[:, -1], linestyle=linestyle,
        #         color=colors['ALL'], label=f'ALL ({t})', lw=lw)
        # PlotSE(ax, ALL_profile, ALL_SE, color=colors['ALL'], factor=factor, min_value=min_value)

        ax.plot(SHALLOW_profile[:, 0], SHALLOW_profile[:, -1], linestyle=linestyle,
                color=colors['SHALLOW'], label=f'SHALLOW ({t})', lw=lw)
        PlotSE(ax, SHALLOW_profile, SHALLOW_SE, color=colors['SHALLOW'], factor=factor, min_value=min_value)

        ax.plot(DEEP_profile[:, 0], DEEP_profile[:, -1], linestyle=linestyle,
                color=colors['DEEP'], label=f'DEEP ({t})', lw=lw)
        PlotSE(ax, DEEP_profile, DEEP_SE, color=colors['DEEP'], factor=factor, min_value=min_value)

    SnapLimitsToTicks([ax], dim='x')
    apply_scientific_notation([ax],decimals=2,scientific=True)
    ax.set_xlabel(r"($kg m^{-3} s^{-1}$)")
    ax.set_ylabel('z (km)')
    ax.grid(True)
    ax.legend(fontsize=8)
    return ax

In [ ]:
#RATIO PLOT
def safe_divide(numerator, denominator):
    # result=numerator/denominator
    # Convert to numpy arrays for safety
    numerator = np.array(numerator)
    denominator = np.array(denominator)
    
    # Mask denominator zeros or very small values to avoid divide-by-zero
    mask = (denominator == 0) | np.isnan(denominator) | np.isnan(numerator)
    
    # Create a result array filled with NaNs initially
    result = np.full_like(numerator, np.nan, dtype=float)
    
    # Only divide where denominator is nonzero and both numerator/denominator are finite
    valid = ~mask
    result[valid] = numerator[valid] / denominator[valid]
    
    return result


def RatioPlot(ax, types, line_styles, type3, type4, factor=1, lw=lw):
    cutoff_height = 100
    # z_cutoff = np.where(CL_ALL_profile_array_G_TO_C[:, 2] <= cutoff_height)[0][-1]
    z_cutoff = None  # or keep the cutoff logic if needed

    colors = {'ALL': 'black', 'SHALLOW': 'green', 'DEEP': 'blue'}

    for t, linestyle in zip(types, line_styles):
        # Reference profiles (G_TO_C)
        prof_ref_all = globals()[f"{t}_ALL_profile_array_G_TO_C_{type3}"]
        prof_ref_shallow = globals()[f"{t}_SHALLOW_profile_array_G_TO_C_{type3}"]
        prof_ref_deep = globals()[f"{t}_DEEP_profile_array_G_TO_C_{type3}"]

        # Target profiles (type3, type4)
        prof_tar_all = globals()[f"{t}_ALL_profile_array_{type3}_{type4}"]
        prof_tar_shallow = globals()[f"{t}_SHALLOW_profile_array_{type3}_{type4}"]
        prof_tar_deep = globals()[f"{t}_DEEP_profile_array_{type3}_{type4}"]

        # Average (and possibly truncate)
        prof_ref_all_avg = averaged_profiles(prof_ref_all)[:z_cutoff]
        prof_ref_sh_avg = averaged_profiles(prof_ref_shallow)[:z_cutoff]
        prof_ref_dp_avg = averaged_profiles(prof_ref_deep)[:z_cutoff]

        prof_tar_all_avg = averaged_profiles(prof_tar_all)[:z_cutoff]
        prof_tar_sh_avg = averaged_profiles(prof_tar_shallow)[:z_cutoff]
        prof_tar_dp_avg = averaged_profiles(prof_tar_deep)[:z_cutoff]

        # # Plot ratio: ref / target (or target / ref, depending on your intent)
        # ax.plot(safe_divide(prof_ref_all_avg[:, 0], prof_tar_all_avg[:, 0]), prof_ref_all_avg[:, -1],
        #         linestyle=linestyle, color=colors['ALL'], label=f'ALL ({t})', lw=lw)

        ax.plot(safe_divide(prof_ref_sh_avg[:, 0], prof_tar_sh_avg[:, 0])*100, prof_ref_sh_avg[:, -1],
                linestyle=linestyle, color=colors['SHALLOW'], label=f'SHALLOW ({t})', lw=lw)

        ax.plot(safe_divide(prof_ref_dp_avg[:, 0], prof_tar_dp_avg[:, 0])*100, prof_ref_dp_avg[:, -1],
                linestyle=linestyle, color=colors['DEEP'], label=f'DEEP ({t})', lw=lw)

    # ax.set_xlim(-0.05, 1.05)
    ax.set_xlim(-5, 105)
    # ax.set_xlabel("Ratio")
    ax.set_xlabel("%")
    ax.set_ylabel("z (km)")
    ax.grid(True)
    ax.axvline(0, linestyle='dashed', color='grey', zorder=-10)
    # ax.axvline(1, linestyle='dashed', color='grey', zorder=-10)
    ax.axvline(100, linestyle='dashed', color='grey', zorder=-10)
    ax.axhline(all_cloudbase, color='purple', linestyle='dashed', zorder=-10)
    ax.axhline(MeanLFC / 1000, color='green', linestyle='dashed', zorder=-10)
    ax.legend(fontsize=8)
    # apply_scientific_notation([ax])
    return ax

In [ ]:
############################################################
#LOADING DATA

In [ ]:
PROCESSING=False
PROCESSING=True

In [ ]:
#CL vs nonCL Tracked Entrainment Profiles
data_type="Tracked_Entrainment_VMF"
type1='CL';type2='nonCL'
dir3=dir+'Project_Algorithms/Tracked_Profiles/OUTPUT_FILES/'
if PROCESSING==True:
    filePath=dir3+f"{data_type}_"+f"{type1}_{type2}_tracked_profiles_{res}_{t_res}_{Np_str}.h5"
elif PROCESSING==False:
    filePath=dir3+f"{data_type}_"+f"{type1}_{type2}_tracked_profiles_{res}_{t_res}_{Np_str}_ORIGINAL.h5"
key_list=[]
with h5py.File(filePath, 'r') as h5f:
    for key in h5f.keys():
        globals()[key] = h5f[key][:]
        if '_squares' not in key:
            key_list.append(key)

#CALCULATING STANDARD DEVIATION
for key in key_list:
    globals()[key+f"_SE"]=ProfileStandardError(globals()[key],globals()[key+f"_squares"]);factor=1.96
    # globals()[key+f"_SE"]=ProfileStandardDeviation(globals()[key],globals()[key+f"_squares"]); factor=1
    # print(key)

In [ ]:
data_type2="Tracked_Combined_Entrainment"
type1='CL';type2='nonCL'
dir3=dir+'Project_Algorithms/Tracked_Profiles/OUTPUT_FILES/'
if PROCESSING==True:
    filePath=dir3 + f"{data_type2}_" + f"{type1}_{type2}_tracked_profiles_{res}_{t_res}_{Np_str}.h5"
elif PROCESSING==False:
    filePath=dir3 + f"{data_type2}_" + f"{type1}_{type2}_tracked_profiles_{res}_{t_res}_{Np_str}_ORIGINAL.h5"
key_list=[]
with h5py.File(filePath, 'r') as h5f:
    for key in h5f.keys():
        globals()[key] = h5f[key][:]
        if '_squares' not in key:
            key_list.append(key)
        # print(key)

#CALCULATING STANDARD DEVIATION
for key in key_list:
    globals()[key+f"_SE"]=ProfileStandardError(globals()[key],globals()[key+f"_squares"]);factor=1.96
    # globals()[key+f"_SE"]=ProfileStandardDeviation(globals()[key],globals()[key+f"_squares"]); factor=1

In [ ]:
#SBZ vs nonSBZ Tracked Entrainment Profiles
data_type="Tracked_Entrainment_VMF"
type1='SBZ';type2='nonSBZ'
dir3=dir+'Project_Algorithms/Tracked_Profiles/OUTPUT_FILES/'
if PROCESSING==True:
    filePath=dir3+f"{data_type}_"+f"{type1}_{type2}_tracked_profiles_{res}_{t_res}_{Np_str}.h5"
elif PROCESSING==False:
    filePath=dir3+f"{data_type}_"+f"{type1}_{type2}_tracked_profiles_{res}_{t_res}_{Np_str}_ORIGINAL.h5"
key_list=[]
with h5py.File(filePath, 'r') as h5f:
    for key in h5f.keys():
        globals()[key] = h5f[key][:]
        if '_squares' not in key:
            key_list.append(key)

#CALCULATING STANDARD DEVIATION
for key in key_list:
    globals()[key+f"_SE"]=ProfileStandardError(globals()[key],globals()[key+f"_squares"]);factor=1.96
    # globals()[key+f"_SE"]=ProfileStandardDeviation(globals()[key],globals()[key+f"_squares"]); factor=1
    # print(key)

In [ ]:
data_type2="Tracked_Combined_Entrainment"
type1='SBZ';type2='nonSBZ'
dir3=dir+'Project_Algorithms/Tracked_Profiles/OUTPUT_FILES/'
if PROCESSING==True:
    filePath=dir3 + f"{data_type2}_" + f"{type1}_{type2}_tracked_profiles_{res}_{t_res}_{Np_str}.h5"
elif PROCESSING==False:
    filePath=dir3 + f"{data_type2}_" + f"{type1}_{type2}_tracked_profiles_{res}_{t_res}_{Np_str}_ORIGINAL.h5"
key_list=[]
with h5py.File(filePath, 'r') as h5f:
    for key in h5f.keys():
        globals()[key] = h5f[key][:]
        if '_squares' not in key:
            key_list.append(key)
        # print(key)

#CALCULATING STANDARD DEVIATION
for key in key_list:
    globals()[key+f"_SE"]=ProfileStandardError(globals()[key],globals()[key+f"_squares"]);factor=1.96
    # globals()[key+f"_SE"]=ProfileStandardDeviation(globals()[key],globals()[key+f"_squares"]); factor=1

In [ ]:
# #ColdPool Tracked Entrainment Profiles
# data_type="Tracked_Entrainment_VMF"
# type2='ColdPool'
# dir3=dir+'Project_Algorithms/Tracked_Profiles/OUTPUT_FILES/'
# if PROCESSING==True:
#     filePath=dir3+f"{data_type}_"+f"{type2}_tracked_profiles_{res}_{t_res}_{Np_str}.h5"
# elif PROCESSING==False:
#     filePath=dir3+f"{data_type}_"+f"{type2}_tracked_profiles_{res}_{t_res}_{Np_str}_ORIGINAL.h5"
# key_list=[]
# with h5py.File(filePath, 'r') as h5f:
#     for key in h5f.keys():
#         globals()[key] = h5f[key][:]
#         if '_squares' not in key:
#             key_list.append(key)

# #CALCULATING STANDARD DEVIATION
# for key in key_list:
#     globals()[key+f"_SE"]=ProfileStandardError(globals()[key],globals()[key+f"_squares"]);factor=1.96
#     # globals()[key+f"_SE"]=ProfileStandardDeviation(globals()[key],globals()[key+f"_squares"]); factor=1
#     # print(key)

# data_type2="Tracked_Combined_Entrainment"
# type2='ColdPool'
# dir3=dir+'Project_Algorithms/Tracked_Profiles/OUTPUT_FILES/'
# if PROCESSING==True:
#     filePath=dir3 + f"{data_type2}_" + f"{type2}_tracked_profiles_{res}_{t_res}_{Np_str}.h5"
# elif PROCESSING==False:
#     filePath=dir3 + f"{data_type2}_" + f"{type2}_tracked_profiles_{res}_{t_res}_{Np_str}_ORIGINAL.h5"
# key_list=[]
# with h5py.File(filePath, 'r') as h5f:
#     for key in h5f.keys():
#         globals()[key] = h5f[key][:]
#         if '_squares' not in key:
#             key_list.append(key)
#         # print(key)

# #CALCULATING STANDARD DEVIATION
# for key in key_list:
#     globals()[key+f"_SE"]=ProfileStandardError(globals()[key],globals()[key+f"_squares"]);factor=1.96
#     # globals()[key+f"_SE"]=ProfileStandardDeviation(globals()[key],globals()[key+f"_squares"]); factor=1

In [ ]:
################################
#PLOTTING

In [ ]:
def SavePlot(fig, type1, res, t_res, Np_str, dir2, limit_y=False):
    # Define output directory
    subdir_name = f'{res}_{t_res}_{Np_str}'
    output_dir = os.path.join(dir2, 'Project_Algorithms', 'Tracked_Profiles', 'PLOTS', 'Tracked_Entrainment', subdir_name)

    if limit_y==True:
        output_dir = os.path.join(output_dir, "SLICE")
    
    # Create the directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    #SAVE PATH
    filename = f'{type1}_Entrainment_Tracked_Profiles_{res}_{t_res}_{Np_str}.jpg'
    save_path = os.path.join(output_dir, filename)
    
    # Save the figure
    fig.savefig(save_path, bbox_inches='tight', dpi=300)
    print(f"Saved figure to: {save_path}")


In [ ]:
fig = plt.figure(figsize=(14, 16))  # Taller to accommodate 3 rows
gs = fig.add_gridspec(3, 3)

types = ['CL', 'nonCL', 'SBZ']
line_styles = ['solid', 'dashed', 'dashdot']

# ===== ROW 1: GENERAL ENTRAINMENT =====
type4 = 'general'
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[0, 2])

PlotProfiles(ax1, types=types, line_styles=line_styles, type3='E', type4=type4[0].upper())
PlotProfiles(ax2, types=types, line_styles=line_styles, type3='D', type4=type4[0].upper())
PlotProfiles(ax3, types=types, line_styles=line_styles, type3='NET', type4=type4[0].upper())

ax1.set_title('Entrainment (General)')
ax2.set_title('Detrainment (General)')
ax3.set_title('Net Entrainment (General)')

# ===== ROW 2: CLOUDY ENTRAINMENT =====
type4 = 'cloudy'
ax4 = fig.add_subplot(gs[1, 0])
ax5 = fig.add_subplot(gs[1, 1])
ax6 = fig.add_subplot(gs[1, 2])

PlotProfiles(ax4, types=types, line_styles=line_styles, type3='E', type4=type4[0].upper())
PlotProfiles(ax5, types=types, line_styles=line_styles, type3='D', type4=type4[0].upper())
PlotProfiles(ax6, types=types, line_styles=line_styles, type3='NET', type4=type4[0].upper())

ax4.set_title('Entrainment (Cloudy)')
ax5.set_title('Detrainment (Cloudy)')
ax6.set_title('Net Entrainment (Cloudy)')

# ===== ROW 3: G vs C Comparison =====
ax7 = fig.add_subplot(gs[2, 0])
ax8 = fig.add_subplot(gs[2, 1])
ax9 = fig.add_subplot(gs[2, 2])

# Replace these with your actual type1/type2 strings
PlotTwo(ax7, types=types, line_styles=line_styles)
# PlotThree(ax7, types=types, line_styles=line_styles)
RatioPlot(ax8, types=types, line_styles=line_styles, type3='E', type4='C')
RatioPlot(ax9, types=types, line_styles=line_styles, type3='D', type4='G')

ax7.set_title('General to Cloudy Updraft Entrainment')
ax8.set_title('General → Cloudy / Cloudy Entrainment')
ax9.set_title('General → Cloudy / General Detrainment')

# # ===== MATCH AXES =====
MatchAxisLimits([ax1, ax2, ax3], dim='x')
MatchAxisLimits([ax4, ax5, ax6, ax7], dim='x')

#LIMIT LEFT X AXIS
for ax in [ax1, ax2, ax4, ax5, ax7]:
    ax.set_xlim(left=0)

#LIMIT Y AXIS
axes=fig.get_axes() 
# # ===== LIMIT Y AXES =====
for ax in axes:
    ax.set_ylim(bottom=0)
    if limit_y==True:
        y_max = 3 if ax in [ax1, ax2, ax3] else 7
        ax.set_ylim(top=y_max)
    elif limit_y==False:
        y_max=20
        ax.set_ylim(top=y_max)

#SET AXVLINES
for ax in axes:
    ax.axvline(0, linestyle='dashed', color='grey', zorder=-10)
    ax.axhline(all_cloudbase, color='purple', linestyle='dashed')
    ax.axhline(MeanLFC / 1000, color='green', linestyle='dashed')

#FIX LEGEND
for ax in [ax1, ax3, ax4, ax5, ax6, ax7, ax8, ax9]:
    ax.legend_.remove() if ax.get_legend() else None  # remove legend if it exists
ax2.legend(loc='upper right',fontsize=8)

fig.subplots_adjust(top=0.92, wspace=0.4, hspace=0.4)

# SAVING FIGURE
SavePlot(fig,'CLnonCL_SBF', res, t_res, Np_str, dir2, limit_y)

In [ ]:
limit_y=False

In [ ]:
####################################
#TESTING

In [ ]:
# #dt=5min
# 1200-750 # 750 km to 1200 km
# # 60m in 60s=1min at 1 m/s
# (1200-750)/(1*60) #takes about 7.5 minutes (a little larger than 1 timestep)
# # ==> maybe cloud base it a little lower in some cases or wind speed is higher (higher than 7.5 m/s)